# Scraping the web II
parse, urlopen

## Yahoo's Finance

In [2]:
from lxml.html import parse
from urllib2 import urlopen

In [4]:
url = "http://finance.yahoo.com/q/op?s=AAPL+Options"

parsed = parse(urlopen(url))
doc = parsed.getroot()

#### Get all links first

In [7]:

links = doc.findall('.//a')
print links[:10]

[<Element a at 0x10581ce10>, <Element a at 0x10581ce68>, <Element a at 0x10581cec0>, <Element a at 0x10581cf18>, <Element a at 0x10581cf70>, <Element a at 0x10581cfc8>, <Element a at 0x105951050>, <Element a at 0x1059510a8>, <Element a at 0x105951100>, <Element a at 0x105951158>]


In [12]:
lnk = links[20]
lnk.get('href')

'http://shopping.yahoo.com/'

In [13]:
lnk.text_content()

'Shopping'

In [15]:
urls =  [lnk.get('href') for lnk in doc.findall('.//a')]
urls[-10:]

['/q/op?s=AAPL&strike=117.00',
 '/q?s=AAPL160422P00117000',
 '/q/op?s=AAPL&strike=118.00',
 '/q?s=AAPL160422P00118000',
 '/q/op?s=AAPL&strike=119.00',
 '/q?s=AAPL160422P00119000',
 '/q/op?s=AAPL&strike=120.00',
 '/q?s=AAPL160422P00120000',
 '/q/op?s=AAPL&strike=125.00',
 '/q?s=AAPL160422P00125000']

### Now, get the tables

In [68]:
tables = doc.findall('.//table')

#print tables[1].text_content()
calls = tables[1]
puts = tables[2]

rows = calls.findall('.//tr')
#print rows[0].text_content()

In [119]:
def _unpack(row, kind='td'):
    elts = row.findall('.//%s' %kind)
    return [val.text_content().replace('\n','') for val in elts]


In [120]:
_unpack(rows[0], kind='th')

[u'                                     Strike                                            \ue004                        \ue002                                                    \u2235 Filter            ',
 'Contract Name',
 u'                                    Last                                            \ue004                        \ue002                                                ',
 u'                                    Bid                                            \ue004                        \ue002                                                ',
 u'                                    Ask                                            \ue004                        \ue002                                                ',
 u'                                    Change                                            \ue004                        \ue002                                                ',
 u'                                    %Change                                  

In [121]:
_unpack(rows[2], kind='td')

['            80.00        ',
 '            AAPL160422C00080000        ',
 '            27.72        ',
 '            27.00        ',
 '            27.30        ',
 '            -1.55        ',
 '                            -5.30%                    ',
 '            1        ',
 '            1        ',
 '            128.13%        ']

### Combining all (using pandas)

In [122]:
from pandas.io.parsers import TextParser

In [123]:
def parse_option_data(table):
    rows = table.findall('.//tr')
    header = _unpack(rows[0], kind ='th')
    data   =[_unpack(r) for r in rows[1:]] 
    return TextParser(data, names=header).get_chunk()

In [124]:
call_data = parse_option_data(calls)

In [126]:
call_data[:10]

,Strike   ∵ Filter,Contract Name,Last  ,Bid  ,Ask  ,Change  ,%Change  ,Volume  ,Open Interest  ,Implied Volatility  
0,...,None,NaN,NaN,NaN,NaN,None,NaN,NaN,None
1,80.00,AAPL160422C00080000,27.72,27.00,27.30,-1.55,-5.30% ...,1.0,1.0,128.13%
2,85.00,AAPL160422C00085000,0.00,16.20,16.75,0.00,0.00% ...,0.0,0.0,0.00%
3,88.00,AAPL160422C00088000,0.00,13.20,14.20,0.00,0.00% ...,0.0,0.0,0.00%
4,89.00,AAPL160422C00089000,0.00,12.20,13.20,0.00,0.00% ...,0.0,0.0,0.00%
5,90.00,AAPL160422C00090000,16.85,17.05,17.25,0.00,0.00% ...,1.0,217.0,80.47%
6,90.50,AAPL160422C00090500,16.55,16.55,16.80,0.00,0.00% ...,14.0,14.0,86.72%
7,91.00,AAPL160422C00091000,18.05,18.90,19.20,0.00,0.00% ...,2.0,2.0,244.14%
8,91.50,AAPL160422C00091500,17.50,18.40,18.70,0.00,0.00% ...,37.0,37.0,239.26%
9,92.00,AAPL160422C00092000,17.38,17.65,18.05,0.00,0.00% ...,2.0,8.0,227.05%
